<summary>Table of Contents</summary>

- [1. Standard Scaler Informer ](#1-standard-scaler-informer)
- [2. Standard Scaler PatchTST](#2-standard-scaler-patchtst)
- [3. MinMax Scaler Informer](#3-minmax-scaler-informer)
- [4. MinMax Scaler PatchTST](#4-minmax-scaler-patchtst)


Here we perform a check on **Italy** dataset to confirm choice of loss function and scaler for our data.

This script is to run the models. Final results are in the notebook "Comparison_IT". 

Please note, the cell content is almost identical. However, when duplicating code and changing some arguments, it becomes easier to store and read results (especially if you want to experiment with 1 subpart) and split long running time into subprocesses. 

In [17]:
import os
import pandas as pd
import subprocess
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df

# 1. Standard Scaler Informer

In [18]:
cuda_device = "0"

# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'IT_data.csv'
losses = ["MSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/standard"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [19]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --dropout 0.1 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type standard \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_96_24_loss_choice_for_IT', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', if_relu=False, channel_mixing=0, seq_len=96, label_len=48, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patien

In [ ]:
path_dir = './results/loss_fnc_choice'
csv_name_scaled = 'informer_loss_functions_results_scaled_IT_default.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_IT_default.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.2293  0.4789  0.3071  0.4386
              2         24        0.2293  0.4789  0.3048  0.4386
              1         96        0.3830  0.6188  0.4100  0.5666
              2         96        0.3659  0.6049  0.4062  0.5539
              1         168       0.4002  0.6326  0.4335  0.5794
              2         168       0.4175  0.6462  0.4400  0.5918
MAE           1         24        0.2362  0.4860  0.2798  0.4451
              2         24        0.2469  0.4969  0.2874  0.4550
              1         96        0.3934  0.6272  0.3986  0.5743
              2         96        0.3954  0.6288  0.3909  0.5757
              1         168       0.4414  0.6644  0.4195  0.6085
              2         168       0.4416  0.6645  0.4241  0.6086

In [21]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        2136431.500  1461.6537   947.5727  0.1027
              2         24        2031066.875  1425.1550   935.1377  0.1001
              1         96        3882534.500  1970.4148  1288.4697  0.1387
              2         96        3475395.250  1864.2412  1267.4960  0.1312
              1         168       4282493.500  2069.4187  1388.6179  0.1458
              2         168       4567560.000  2137.1851  1427.0869  0.1505
MAE           1         24        1555223.500  1247.0861   786.8116  0.0876
              2         24        1826092.875  1351.3301   822.8324  0.0950
              1         96        3858870.000  1964.4006  1243.5869  0.1382
              2         96        3485771.250  1867.0220  1180.4133  0.1314
              1         168       4175248.250  2043.3424  1301.3749  0.1439
              2         168       4408365.500  2099.6108  1332.4801  0.1479

In [ ]:
# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.2415  0.4914  0.2836  0.4501
         MSE            0.2293  0.4789  0.3060  0.4386
96       MAE            0.3944  0.6280  0.3947  0.5750
         MSE            0.3744  0.6119  0.4081  0.5602
168      MAE            0.4415  0.6644  0.4218  0.6085
         MSE            0.4089  0.6394  0.4367  0.5856

In [23]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.690658e+06  1299.2081   804.8220  0.0913
         MSE            2.083749e+06  1443.4044   941.3552  0.1014
96       MAE            3.672321e+06  1915.7113  1212.0001  0.1348
         MSE            3.678965e+06  1917.3280  1277.9828  0.1349
168      MAE            4.291807e+06  2071.4766  1316.9275  0.1459
         MSE            4.425027e+06  2103.3019  1407.8524  0.1482

# 2. Standard Scaler PatchTST

In [24]:
# Dynamic + default variables
pred_lens = ["24", "96", "168"]
seq_len = "336"
lr = "0.0001"
model = "PatchTST"
itr = 2  
n_heads = "16"
d_model = "128"
d_ff = "256"
dropout = "0.2"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers 3 \
              --factor 1 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --patch_len 32 \
              --stride 16 \
              --overlapping_windows \
              --inverse \
              --scaler_type standard \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_336_24_loss_choice_for_IT', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='standard', if_relu=False, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=32, stride=16, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, pati

In [ ]:
path_dir = './results/loss_fnc_choice'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_IT_default.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_IT_default.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.2016  0.4490  0.2636  0.4112
              2         24        0.2025  0.4500  0.2619  0.4122
              1         96        0.3570  0.5975  0.3740  0.5470
              2         96        0.3597  0.5998  0.3818  0.5491
              1         168       0.3990  0.6316  0.4067  0.5785
              2         168       0.3850  0.6205  0.4035  0.5683
MAE           1         24        0.2029  0.4505  0.2528  0.4126
              2         24        0.2046  0.4524  0.2532  0.4143
              1         96        0.3803  0.6167  0.3621  0.5647
              2         96        0.3867  0.6219  0.3646  0.5694
              1         168       0.4169  0.6457  0.3925  0.5914
              2         168       0.4201  0.6481  0.3892  0.5936

In [26]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1205259.875  1097.8433   717.0481  0.0771
              2         24        1178370.250  1085.5276   706.2290  0.0763
              1         96        2492006.000  1578.6089  1051.8491  0.1111
              2         96        2693139.250  1641.0787  1101.3221  0.1155
              1         168       3082198.250  1755.6190  1179.6866  0.1237
              2         168       3154101.000  1775.9789  1187.3914  0.1251
MAE           1         24        1151100.125  1072.8933   668.9365  0.0754
              2         24        1154194.750  1074.3346   667.5917  0.0755
              1         96        2383695.250  1543.9220   980.0680  0.1087
              2         96        2395718.250  1547.8108   982.5762  0.1089
              1         168       3020437.250  1737.9406  1113.3966  0.1224
              2         168       2837935.750  1684.6174  1074.3492  0.1187

In [ ]:
# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.2038  0.4514  0.2530  0.4134
         MSE            0.2021  0.4495  0.2628  0.4117
96       MAE            0.3835  0.6193  0.3633  0.5670
         MSE            0.3583  0.5986  0.3779  0.5481
168      MAE            0.4185  0.6469  0.3908  0.5925
         MSE            0.3920  0.6261  0.4051  0.5734

In [28]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.152647e+06  1073.6140   668.2641  0.0754
         MSE            1.191815e+06  1091.6854   711.6385  0.0767
96       MAE            2.389707e+06  1545.8664   981.3221  0.1088
         MSE            2.592573e+06  1609.8438  1076.5856  0.1133
168      MAE            2.929186e+06  1711.2790  1093.8729  0.1205
         MSE            3.118150e+06  1765.7990  1183.5390  0.1244

In [29]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'standard_unscaled_IT')

# 3. MinMax Scaler Informer

We can use now "ReLU" activation function due to MinMax Scaler.

With BS 1036, ReLU - results are bad. (as twice as bad as with 32!)

In [30]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'IT_data.csv'
losses = ["MSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/min_max"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [31]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 48 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --dropout 0.1 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_96_24_loss_choice_for_IT', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=96, label_len=48, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.1, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patience=

	iters: 400, epoch: 1 | loss: 0.0614750
	speed: 0.0340s/iter; left time: 602.7891s
	iters: 500, epoch: 1 | loss: 0.0473126
	speed: 0.0339s/iter; left time: 597.8125s
	iters: 600, epoch: 1 | loss: 0.0451944
	speed: 0.0339s/iter; left time: 593.9310s
	iters: 700, epoch: 1 | loss: 0.0370400
	speed: 0.0339s/iter; left time: 590.5924s
	iters: 800, epoch: 1 | loss: 0.0355196
	speed: 0.0339s/iter; left time: 587.0480s
	iters: 900, epoch: 1 | loss: 0.0400502
	speed: 0.0339s/iter; left time: 583.7089s
-------------------------------------------------------------------------------------
Epoch: 1
Cost time: 00h:00m:31.55s
Steps: 906 | Train Loss: 0.0606730 Vali Loss: 0.0234355 Test Loss: 0.0250223
Validation loss decreased (inf --> 0.023436).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.0261067
	speed: 0.1125s/iter; left time: 1924.9819s
	iters: 200, epoch: 2 | loss: 0.0200592
	speed: 0.0338s/iter; left time: 575.7650s
	iters: 300, epoch: 2 | loss: 0.0135396


In [ ]:
path_dir = './results/choice'
csv_name_scaled = 'informer_loss_functions_results_scaled_minmax_IT_default.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_minmax_IT_default.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0111  0.1055  0.0649  0.3989
              2         24        0.0115  0.1073  0.0672  0.4054
              1         96        0.0195  0.1395  0.0926  0.5276
              2         96        0.0187  0.1366  0.0904  0.5165
              1         168       0.0212  0.1455  0.0975  0.5507
              2         168       0.0205  0.1433  0.0957  0.5423
MAE           1         24        0.0120  0.1097  0.0642  0.4145
              2         24        0.0131  0.1144  0.0672  0.4322
              1         96        0.0223  0.1494  0.0913  0.5648
              2         96        0.0193  0.1388  0.0861  0.5248
              1         168       0.0208  0.1442  0.0944  0.5457
              2         168       0.0234  0.1530  0.0991  0.5788

In [33]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1800234.625  1341.7283   865.4649  0.0943
              2         24        1954150.750  1397.9094   902.0552  0.0982
              1         96        4252090.500  2062.0598  1342.6979  0.1451
              2         96        3400539.250  1844.0551  1237.2542  0.1298
              1         168       4480882.500  2116.8096  1392.9763  0.1491
              2         168       4692489.500  2166.2156  1400.1951  0.1526
MAE           1         24        1767036.250  1329.2992   813.9424  0.0934
              2         24        1968128.250  1402.8999   857.3510  0.0986
              1         96        3587957.250  1894.1904  1209.4962  0.1333
              2         96        3035685.000  1742.3218  1125.7732  0.1226
              1         168       4198082.000  2048.9221  1319.8297  0.1443
              2         168       5321987.500  2306.9434  1452.3975  0.1625

In [ ]:
# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0126  0.1120  0.0657  0.4234
         MSE            0.0113  0.1064  0.0660  0.4021
96       MAE            0.0208  0.1441  0.0887  0.5448
         MSE            0.0191  0.1381  0.0915  0.5220
168      MAE            0.0221  0.1486  0.0967  0.5623
         MSE            0.0209  0.1444  0.0966  0.5465

In [35]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.867582e+06  1366.0995   835.6467  0.0960
         MSE            1.877193e+06  1369.8188   883.7600  0.0963
96       MAE            3.311821e+06  1818.2561  1167.6347  0.1280
         MSE            3.826315e+06  1953.0574  1289.9760  0.1374
168      MAE            4.760035e+06  2177.9327  1386.1136  0.1534
         MSE            4.586686e+06  2141.5126  1396.5857  0.1508

# 4. MinMax Scaler PatchTST

In [36]:
log_dir = f"logs/loss_choice/min_max"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

In [37]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "336"
lr = "0.0001"
model = "PatchTST"
itr = 2 
n_heads = "16"
d_model = "128"
d_ff = "256"
dropout = "0.2"
os.environ["CUDA_VISIBLE_DEVICES"] = cuda_device

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --pred_len {pred_len} \
              --e_layers 3 \
              --factor 1 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 20 \
              --patience 5 \
              --n_heads {n_heads} \
              --d_model {d_model} \
              --d_ff {d_ff} \
              --dropout {dropout} \
              --fc_dropout {dropout} \
              --patch_len 32 \
              --stride 16 \
              --overlapping_windows \
              --inverse \
              --scaler_type minmax \
              --if_relu \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_336_24_loss_choice_for_IT', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work-1/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='minmax', if_relu=True, channel_mixing=0, seq_len=336, label_len=48, pred_len=24, inverse=True, loss_fnc='MSE', fc_dropout=0.2, head_dropout=0.0, patch_len=32, stride=16, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=128, n_heads=16, e_layers=3, d_layers=1, d_ff=256, moving_avg=25, factor=1, distil=True, dropout=0.2, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=20, batch_size=32, patienc

	iters: 300, epoch: 4 | loss: 0.0091413
	speed: 0.0092s/iter; left time: 137.9772s
	iters: 400, epoch: 4 | loss: 0.0086968
	speed: 0.0092s/iter; left time: 136.6427s
	iters: 500, epoch: 4 | loss: 0.0107999
	speed: 0.0092s/iter; left time: 135.5690s
	iters: 600, epoch: 4 | loss: 0.0097379
	speed: 0.0092s/iter; left time: 135.0998s
	iters: 700, epoch: 4 | loss: 0.0107894
	speed: 0.0090s/iter; left time: 131.9590s
	iters: 800, epoch: 4 | loss: 0.0073125
	speed: 0.0089s/iter; left time: 129.4746s
-------------------------------------------------------------------------------------
Epoch: 4
Cost time: 00h:00m:08.53s
Steps: 899 | Train Loss: 0.0096880 Vali Loss: 0.0092516 Test Loss: 0.0103331
Validation loss decreased (0.009379 --> 0.009252).  Saving model ...
Updating learning rate to 9e-05
	iters: 100, epoch: 5 | loss: 0.0096844
	speed: 0.0415s/iter; left time: 593.1690s
	iters: 200, epoch: 5 | loss: 0.0078026
	speed: 0.0095s/iter; left time: 134.6578s
	iters: 300, epoch: 5 | loss: 0.00860

In [ ]:
path_dir = './results/loss_fnc_choice'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_minmax_IT_default.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_minmax_IT_default.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
#patchtst_df_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.0101  0.1005  0.0586  0.3798
              2         24        0.0102  0.1009  0.0581  0.3812
              1         96        0.0180  0.1341  0.0824  0.5071
              2         96        0.0179  0.1340  0.0839  0.5065
              1         168       0.0200  0.1413  0.0891  0.5347
              2         168       0.0193  0.1388  0.0888  0.5250
MAE           1         24        0.0102  0.1008  0.0564  0.3808
              2         24        0.0102  0.1009  0.0565  0.3813
              1         96        0.0187  0.1368  0.0808  0.5173
              2         96        0.0190  0.1378  0.0810  0.5209
              1         168       0.0211  0.1453  0.0873  0.5499
              2         168       0.0209  0.1446  0.0878  0.5471

In [39]:
patchtst_results_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1228456.750  1108.3577   708.9411  0.0779
              2         24        1194382.000  1092.8778   695.2028  0.0768
              1         96        2541549.750  1594.2239  1034.8856  0.1122
              2         96        2719271.750  1649.0215  1078.8997  0.1160
              1         168       3073615.000  1753.1729  1146.7260  0.1235
              2         168       3200891.750  1789.1036  1168.5704  0.1260
MAE           1         24        1136971.750  1066.2888   656.8383  0.0749
              2         24        1138836.625  1067.1628   654.0407  0.0750
              1         96        2374954.250  1541.0886   972.7524  0.1085
              2         96        2350334.250  1533.0800   967.2764  0.1079
              1         168       2851058.500  1688.5078  1070.8998  0.1189
              2         168       3054074.250  1747.5909  1107.2745  0.1231

In [ ]:
# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.0102  0.1008  0.0564  0.3811
         MSE            0.0101  0.1007  0.0583  0.3805
96       MAE            0.0188  0.1373  0.0809  0.5191
         MSE            0.0180  0.1340  0.0831  0.5068
168      MAE            0.0210  0.1450  0.0876  0.5485
         MSE            0.0196  0.1400  0.0889  0.5299

In [41]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.137904e+06  1066.7258   655.4395  0.0750
         MSE            1.211419e+06  1100.6177   702.0720  0.0773
96       MAE            2.362644e+06  1537.0843   970.0144  0.1082
         MSE            2.630411e+06  1621.6227  1056.8926  0.1141
168      MAE            2.952566e+06  1718.0494  1089.0872  0.1210
         MSE            3.137253e+06  1771.1382  1157.6482  0.1248

In [42]:
# Rename folders
new_path_name = 'minmax_IT'
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line
os.rename("results_loss_unscaled", new_path_name)